# TITLE: 

# By 
## Name1:
## Name2:
## Name3:

In [1]:
## Libaries
import pandas as pd
import numpy as np
import plotly.express as px
import pytz # get timezone inf
import matplotlib.pyplot as plt
import datetime


In [2]:
# Files
API_GameLogs = 'gameLogs.csv'
data = pd.read_csv(API_GameLogs)

data.tail()


# read NBA dataset, General Information
game_data = pd.read_csv('Datasets/NBA Database Datasets/game.csv')
players = pd.read_csv('Datasets/NBA Database Datasets/player.csv')
team_data = pd.read_csv('Datasets/NBA Database Datasets/team.csv')



# Basics Graphs

In [17]:
import pandas as pd
import altair as alt


# Sample code to prepare the game_data_filtered if you've not already:
game_data_filtered = pd.read_csv('Datasets/NBA Database Datasets/game.csv')

game_data_filtered['year'] = pd.to_datetime(game_data_filtered['game_date']).dt.year

# Group by stats for home and away teams combined
tov_data = game_data_filtered.groupby('year').agg({
    'tov_home': 'mean',
    'tov_away': 'mean'
}).reset_index().melt('year', var_name='stat', value_name='tov_value')

fg_pct_data = game_data_filtered.groupby('year').agg({
    'fg_pct_home': 'mean',
    'fg_pct_away': 'mean'
}).reset_index().melt('year', var_name='stat', value_name='fg_pct_value')

reb_home_data = game_data_filtered.groupby('year').agg({
    'reb_home': 'mean',
    'reb_away': 'mean'
}).reset_index().melt('year', var_name='stat', value_name='reb_value')

oreb_data = game_data_filtered.groupby('year').agg({
    'oreb_home': 'mean',
    'oreb_away': 'mean'
}).reset_index().melt('year', var_name='stat', value_name='oreb_value')


# Create a slider for year selection
slider = alt.binding_range(min=2000, max=tov_data['year'].max(), step=1, name='Year:')
slider_selection = alt.selection_single(bind=slider, fields=['year'], name='Selector')

# graphs for tov, fg_pct, reb, oreb

tov_Chart = alt.Chart(tov_data).mark_bar(opacity=0.7).encode(
    x=alt.X('stat:N', axis=alt.Axis(title='Turnovers')),
    y=alt.Y('tov_value:Q', axis=alt.Axis(title='Value')),
    tooltip=['year', 'stat', 'tov_value']
).properties(
    width=200
).add_selection(
    slider_selection
).transform_filter(
    slider_selection
)

fg_pct_Chart = alt.Chart(fg_pct_data).mark_bar(opacity=0.7).encode(
    x=alt.X('stat:N', axis=alt.Axis(title='Field Goal Percentage')),
    y=alt.Y('fg_pct_value:Q', axis=alt.Axis(title='Value')),
    tooltip=['year', 'stat', 'fg_pct_value']
).properties(
    width=200
).add_selection(
    slider_selection
).transform_filter(
    slider_selection
)

reb_Chart = alt.Chart(reb_home_data).mark_bar(opacity=0.7).encode(
    x=alt.X('stat:N', axis=alt.Axis(title='Rebounds')),
    y=alt.Y('reb_value:Q', axis=alt.Axis(title='Value')),
    tooltip=['year', 'stat', 'reb_value']
).properties(
    width=200
).add_selection(
    slider_selection
).transform_filter(
    slider_selection
)

oreb_Chart = alt.Chart(oreb_data).mark_bar(opacity=0.7).encode(
    x=alt.X('stat:N', axis=alt.Axis(title='Offesive Rebounds')),
    y=alt.Y('oreb_value:Q', axis=alt.Axis(title='Value')),
    tooltip=['year', 'stat', 'oreb_value']
).properties(
    width=200
).add_selection(
    slider_selection
).transform_filter(
    slider_selection
)
 
# Combine the charts
combined_charts = alt.hconcat(tov_Chart, fg_pct_Chart, reb_Chart, oreb_Chart).resolve_scale(y='independent').properties(title='Average Game Statistics per Year')

# Display the chart
combined_charts


/Users/apple/miniconda3/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/apple/miniconda3/lib/python3.11/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/Users/apple/miniconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/Users/apple/miniconda3/lib/python3.11/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()``

alt.HConcatChart(...)

In [4]:
# Filter data for the Atlanta Hawks
hawks_data = data[data['NICKNAME'] == 'Hawks']

# Convert 'GAME_DATE' to datetime for better plotting
hawks_data['GAME_DATE'] = pd.to_datetime(hawks_data['GAME_DATE'])

# Sort the DataFrame by 'GAME_DATE' to ensure correct chronological plotting
hawks_data_sorted = hawks_data.sort_values(by='GAME_DATE')

# Plotting total win percentage over time, using the sorted DataFrame
fig = px.scatter(hawks_data_sorted, x='GAME_DATE', y='TOTAL_WIN_PCTG', title='Atlanta Hawks Total Win Percentage Over Time',
              labels={'TOTAL_WIN_PCTG': 'Total Win Percentage', 'GAME_DATE': 'Game Date'}, trendline='lowess')

fig.update_layout(xaxis_title='Game Date', yaxis_title='Total Win %', showlegend=False)
fig.show()


/var/folders/wj/_8fkb1x10xn9hxrq5p8pfh280000gn/T/ipykernel_72247/2460080567.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hawks_data['GAME_DATE'] = pd.to_datetime(hawks_data['GAME_DATE'])
/Users/apple/miniconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result
  v = v.dt.to_pydatetime()
/Users/apple/miniconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning: The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will retur

In [5]:
# Scatter plot for shooting efficiency
fig = px.scatter(hawks_data_sorted, x='FGA', y='FG', trendline='ols', 
                 title='Field Goals Made vs. Attempted (Shooting Efficiency)',
                 labels={'FG': 'Field Goals Made', 'FGA': 'Field Goals Attempted'})

fig.update_layout(xaxis_title='Field Goals Attempted', yaxis_title='Field Goals Made', showlegend=False)
fig.show()


In [6]:
# Line plot for total rebounds over time
fig = px.line(hawks_data_sorted, x='GAME_DATE', y='TOT_REB', 
              title='Total Rebounds Over Time',
              labels={'TOT_REB': 'Total Rebounds', 'GAME_DATE': 'Game Date'})

fig.update_layout(xaxis_title='Game Date', yaxis_title='Total Rebounds', showlegend=False)
fig.show()


/Users/apple/miniconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [7]:
# Scatter plot to observe the impact of rest days on points scored
fig = px.scatter(hawks_data_sorted, x='NUM_REST_DAYS', y='PTS', trendline='ols',
                 title='Impact of Rest Days on Points Scored',
                 labels={'NUM_REST_DAYS': 'Number of Rest Days', 'PTS': 'Points Scored'})

fig.update_layout(xaxis_title='Number of Rest Days', yaxis_title='Points Scored', showlegend=False)
fig.show()


In [8]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Create a figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=hawks_data_sorted['GAME_DATE'], y=hawks_data_sorted['TOTAL_WIN_PCTG'], name="Total Win Percentage"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=hawks_data_sorted['GAME_DATE'], y=hawks_data_sorted['ROLLING_OE'], name="Offensive Efficiency"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(title_text="Total Win Percentage and Offensive Efficiency Over Time")

# Set x-axis title
fig.update_xaxes(title_text="Game Date")

# Set y-axes titles
fig.update_yaxes(title_text="Total Win Percentage", secondary_y=False)
fig.update_yaxes(title_text="Offensive Efficiency", secondary_y=True)

fig.show()


/Users/apple/miniconda3/lib/python3.11/site-packages/_plotly_utils/basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



# Model